## Cálculo de pi secuencial

In [1]:
#include <stdio.h>
#include <stdlib.h>

#define ITERATIONS 2e09

int calculatePi(double *piTotal){   
    int i = 0;
    do{
        *piTotal = *piTotal + (double)(4.0 / ((i*2)+1));
        i++;
        *piTotal = *piTotal - (double)(4.0 / ((i*2)+1));
        i++;
    }while(i < ITERATIONS);
    return 0;
}


int main()
{
    double pi;
    pi = 0;
    calculatePi(&pi);
    printf("\npi: %2.12f \n", pi);
}


pi: 3.141592658505 


## Cálculo de pi con dos procesos

In [2]:
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>

#define ITERATIONS 2e09

int calculatePi(double *piTotal, int initIteration, int endIteration)
{   int i = initIteration;
    
    do{
        *piTotal = *piTotal + (double)(4.0 / ((i*2)+1));
        i++;
        *piTotal = *piTotal - (double)(4.0 / ((i*2)+1));
        i++;
    }while(i < endIteration);

    return 0;
}


int main(){
    
    double pi, pi_hijo;
    int pipefd[2], r;
    pid_t pid;
    
    r = pipe(pipefd);
    if(r < 0){
        perror("error pipe: ");
        exit(-1);
    }
    pi = 0;

    pid = fork();
    if(pid < 0){
        perror("Error en fork ");
        exit(-1);
    } 

    if(pid == 0){  //hijo
        close(pipefd[0]);
        calculatePi(&pi, 0, (ITERATIONS/2) );
        r = write(pipefd[1], &pi, sizeof(double));
        if(r <= 0){
            perror("error write: ");
            exit(-1);
        }        
        close(pipefd[1]);
        exit(0);
    }else{   
        close(pipefd[1]);
        calculatePi(&pi, (ITERATIONS/2), ITERATIONS);
        r = read(pipefd[0], &pi_hijo, sizeof(double));
        if(r <= 0){
            perror("error read: ");
            exit(-1);
        }
        close(pipefd[0]);
        pi = pi + pi_hijo;
    }

    printf("\npi: %2.12f \n", pi);
    return 0;
}





pi: 3.141592658506 


## Cálculo de pi con hilos posix

In [4]:
//%cflags:-lpthread -lm

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <pthread.h>

#define ITERATIONS 2e09
#define THREADS 8

double piTotal[THREADS];

void *calculatePi(void *arg)
{
  int initIteration, endIteration, threadId = *(int *)arg;

  initIteration = (ITERATIONS/THREADS) * threadId;
  endIteration = initIteration + ((ITERATIONS/THREADS) - 1);
  piTotal[threadId] = 0.0;
    do{
        piTotal[threadId] = piTotal[threadId] + (double)(4.0 / ((initIteration*2)+1));
        initIteration++;
        piTotal[threadId] = piTotal[threadId] - (double)(4.0 / ((initIteration*2)+1));
        initIteration++;
    }while(initIteration < endIteration);

    return 0;
}

int main()
{
    int threadId[THREADS], i, *retval;
    pthread_t thread[THREADS];

    for(i = 0; i < THREADS; i++){
        threadId[i] = i;
        pthread_create(&thread[i], NULL, (void *)calculatePi, &threadId[i]);
    
    }

    for(i = 0; i < THREADS; i++){
        pthread_join(thread[i], (void **)&retval);
    }

    for(i = 1; i < THREADS; i++){
        piTotal[0] = piTotal[0] + piTotal[i];
    }

    printf("\npi: %2.12f \n", piTotal[0]);
}



pi: 3.141592658507 
